# AutoCodificadores para la reducción de dimensionaliad

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px 

### Obtención de datos

En esta ocasión se usará el método **make_blobs** para generar datos correspondientes a dos clases

In [ ]:
from sklearn.datasets import make_blobs

In [ ]:
datos = make_blobs(n_samples = 300,  # Cuántas observaciones queremos
    n_features = 2,                  # Cuántas variables queremos
    centers = 2,                     # Cuántos grupos o clusters queremos
    cluster_std = 1.0,               # Qué variabilidad queremos
    random_state = 101)              # Para reproducibilidad

In [ ]:
datos

In [ ]:
X, y = datos

### Para hacerlo un poco más interesante, se agrega "ruido" a los datos

#### Se generan

In [ ]:
np.random.seed(seed = 101)
z_ruido = np.random.normal(size = len(X))
z_ruido = pd.Series(z_ruido)

#### Se agrega el ruido a los datos (features)

In [ ]:
features = pd.DataFrame(X)
features = pd.concat([features, z_ruido],
                     axis = 1)
features.columns = ['X1', 'X2', 'X3']

In [ ]:
features.head()

### Visualización de los datos

In [ ]:
# Utilizando MatPlotLib
plt.scatter(features['X1'], 
            features['X2'],
            c = y) # Usar la y para color

In [ ]:
# Utilizando Plotly Express
fig = px.scatter(x = features['X1'], 
                 y = features['X2'], 
                 color = y,
                 width = 600,
                 height = 500)
fig.show()

### Para graficar con MatPlotLib en 3-D

https://matplotlib.org/mpl_toolkits/mplot3d/tutorial.html#scatter-plots

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
# Si se tiene la última versión de Jupyter NB
#%matplotlib notebook

%matplotlib inline

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(features['X1'],
           features['X2'],
           features['X3'],
           c = y)

### 3D con Plotly Express

Si se gira para visualizar bien los dos clusters, se vé que el ruido (z) no discrimina los dos grupos, más o menos tiene el mismo rango

In [ ]:
fig = px.scatter_3d(x = features['X1'], 
                    y = features['X2'], 
                    z = features['X3'],
                    color = y,
                    width = 600, height = 500)
fig.show()

# Codificador y Decodificador

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# 3 --> 2
codificador = Sequential()
codificador.add(Dense(units = 2,
                      activation = 'relu',
                      input_shape = [3]))

In [ ]:
# 2 ---> 3
decodificador = Sequential()
decodificador.add(Dense(units = 3,
                        activation = 'relu',
                        input_shape = [2]))

### Ahora se crea el autocodificador completo

#### Se utilizará el optimizador *Stochastic Gradient Descent* porque permite "jugar" con la tasa de aprendizaje *lr* (learning rate)

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
# Autocodificador
# 3 ---> 2 ----> 3
autocodificador = Sequential([codificador,
                              decodificador])

### Compilación del Autocodificador

In [ ]:
autocodificador.compile(loss = "mse",
                        optimizer = SGD(lr = 1.5))

### Normalización de los datos

#### Nota:  se usarán todos los datos!  No tiene sentido usar datos de entrenamiento y de prueba ya que no hay una respuesta correcta en esto.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
normalizador = MinMaxScaler()
datos_normalizados = normalizador.fit_transform(features)

In [ ]:
datos_normalizados

### Entrenamiento del Autocodificador

#### Nota:  La entrada y la salida son lo mismo

In [ ]:
autocodificador.fit(datos_normalizados,
                    datos_normalizados,
                    epochs = 5)

### Si se quieren ver las dos dimensiones a las que redujo el codificador

In [ ]:
codificado_2dim = codificador.predict(datos_normalizados)

In [ ]:
codificado_2dim.shape

In [ ]:
codificado_2dim

In [ ]:
plt.scatter(codificado_2dim[:, 0],
            codificado_2dim[:, 1],
            c = y)

In [ ]:
fig = px.scatter(x = codificado_2dim[:, 0], 
                 y = codificado_2dim[:, 1], 
                 color = y,
                 width = 700,
                 height = 500)
fig.show()